# Poisson in 1D with scikit-fem

We solve the boundary value problem
$$-u''(x)=1,\quad x\in(0,1),\qquad u(0)=u(1)=0.$$

**Exact solution:** $u(x)=\tfrac{1}{2}x(1-x)$.
We use a P1 (linear) finite element discretization on a uniform 1D mesh.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from skfem import MeshLine, Basis, ElementLineP1, asm
from skfem.models.poisson import laplace, unit_load

# Mesh and basis on [0, 1]
refine = 6   # 64 elements
m = MeshLine(np.linspace(0, 1, 2**refine + 1))
e = ElementLineP1()
basis = Basis(m, e)

# Assemble stiffness and load for -u'' = 1
A = asm(laplace, basis)
b = asm(unit_load, basis)

# Find boundary nodes (x=0 and x=1)
x = m.p[0]
boundary = np.where((np.abs(x) < 1e-12) | (np.abs(x - 1.0) < 1e-12))[0]

# Apply Dirichlet BCs: convert to dense and modify
A_dense = A.toarray()
for i in boundary:
    A_dense[i, :] = 0
    A_dense[i, i] = 1
    b[i] = 0

# Solve the system
u = np.linalg.solve(A_dense, b)

# Exact solution and error
u_exact = 0.5 * x * (1 - x)
err_inf = np.max(np.abs(u - u_exact))
print(f"Infinity-norm error: {err_inf:.3e}")

# Plot FE vs exact
plt.figure(figsize=(6,4))
plt.plot(x, u, marker='o', label='FE (P1)')
plt.plot(x, u_exact, linestyle='--', label='Exact 0.5·x·(1-x)')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.title('1D Poisson: -u"=1 on (0,1), u(0)=u(1)=0')
plt.legend()
plt.tight_layout()
plt.show()

### Experiments
- Change `refine` and observe the error decrease.
- Replace the RHS with a function by defining a custom load form.
- Try nonzero Dirichlet data by modifying the condensation step.